## Gradio Pycaret Classification

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('stroke_data.csv')

In [2]:
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
1,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
2,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
3,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
4,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
...,...,...,...,...,...,...,...,...,...,...,...
29060,Female,10.0,0,0,No,children,Urban,58.64,20.4,never smoked,0
29061,Female,56.0,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
29062,Female,82.0,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
29063,Male,40.0,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


In [3]:
df.dtypes

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [4]:
df['stroke'] = pd.Categorical(df.stroke)

In [5]:
df.dtypes

gender                 object
age                   float64
hypertension            int64
heart_disease           int64
ever_married           object
work_type              object
Residence_type         object
avg_glucose_level     float64
bmi                   float64
smoking_status         object
stroke               category
dtype: object

In [29]:
df.work_type.value_counts()

Private          18950
Self-employed     5204
Govt_job          4195
children           615
Never_worked       101
Name: work_type, dtype: int64

In [25]:
df.hypertension.value_counts()

0    25824
1     3241
Name: hypertension, dtype: int64

In [26]:
df.heart_disease.value_counts()

0    27549
1     1516
Name: heart_disease, dtype: int64

In [27]:
df.ever_married.value_counts()

Yes    21687
No      7378
Name: ever_married, dtype: int64

In [28]:
df.Residence_type.value_counts()

Urban    14592
Rural    14473
Name: Residence_type, dtype: int64

In [6]:
# initialize setup
from pycaret.classification import *
s = setup(df, target = 'stroke', session_id = 123)

,Description,Value
0,Session id,123
1,Target,stroke
2,Target type,Binary
3,Original data shape,"(29065, 11)"
4,Transformed data shape,"(29065, 17)"
5,Transformed train set shape,"(20345, 17)"
6,Transformed test set shape,"(8720, 17)"
7,Ordinal features,3
8,Numeric features,5
9,Categorical features,5


In [7]:
best = compare_models(n_select = 15)
compare_model_results = pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9811,0.8343,0.0000,0.0000,0.0000,0.0000,0.0000,2.2290
ridge,Ridge Classifier,0.9811,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2160
dummy,Dummy Classifier,0.9811,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1230
svm,SVM - Linear Kernel,0.9810,0.0000,0.0026,0.0250,0.0048,0.0044,0.0076,0.3600
ada,Ada Boost Classifier,0.9810,0.8134,0.0000,0.0000,0.0000,-0.0002,-0.0004,0.3770
rf,Random Forest Classifier,0.9809,0.7454,0.0000,0.0000,0.0000,-0.0005,-0.0015,0.4090
knn,K Neighbors Classifier,0.9807,0.5813,0.0026,0.1000,0.0050,0.0041,0.0138,0.6180
gbc,Gradient Boosting Classifier,0.9805,0.8247,0.0079,0.1667,0.0149,0.0132,0.0314,0.7650
lightgbm,Light Gradient Boosting Machine,0.9805,0.7958,0.0026,0.0500,0.0050,0.0036,0.0085,0.3160
catboost,CatBoost Classifier,0.9805,0.7860,0.0026,0.1000,0.0051,0.0038,0.0134,10.7380


Processing:   0%|          | 0/79 [00:00<?, ?it/s]

In [8]:
len(best)

15

In [9]:
print(best[:5])

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=123, solver='auto',
                tol=0.0001), DummyClassifier(constant=None, random_state=123, strategy='prior'), SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=123, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=

In [31]:
# creating a predict function to be passed into gradio UI
def predict(model, age, hypertension, heart_disease, avg_glucose_level, bmi):
    # Create a DataFrame for the input features
    df = pd.DataFrame.from_dict({
        'age': [age],
        'hypertension': [hypertension],
        'heart_disease': [heart_disease],
        'avg_glucose_level': [avg_glucose_level],
        'bmi': [bmi],
    })
    
    model_index = list(compare_model_results['Model']).index(model)
    model = best[model_index]
    pred = predict_model(model, df, raw_score=True)
    return {'0': pred['Score_0'][0], '1': pred['Score_1'][0]}

In [30]:
df.work_type.value_counts()

Private          18950
Self-employed     5204
Govt_job          4195
children           615
Never_worked       101
Name: work_type, dtype: int64

In [32]:
import gradio as gr

model = gr.Dropdown(list(compare_model_results['Model']), label="Model")
age = gr.Slider(minimum=10, maximum=100, value=50, label='Age')
hypertension = gr.Radio([0, 1], label='Hypertension')
heart_disease = gr.Radio([0, 1], label='Heart Disease')
avg_glucose_level = gr.Slider(minimum=55, maximum=300, value=150, label='Avg Glucose Level')
bmi = gr.Slider(minimum=10, maximum=100, value=25, label='BMI')

# Launch the interface
gr.Interface(predict, [model, age, hypertension, heart_disease, avg_glucose_level, bmi], outputs="label", live=True).launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-p

Traceback (most recent call last):
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-p

Traceback (most recent call last):
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-p